# Parameters List




In [1]:
import numpy as np
k_batch=list(np.arange(0.5, 4.5, 0.25))
count_threshold_measure=['median','max']
logical_parameter=['and','or']

In [2]:
import itertools

hyperparameter_space = {
    'k_batch': k_batch,
    'count_threshold_measure':count_threshold_measure,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))


In [3]:
all_combinations;

In [4]:
len(all_combinations)

64

### Design the Process flow for grid search

1. Autoencoder is pre-trained , which will be used for all combinations ,this will reduce the processing overhead
2. The validation data needs to be processed once 
3. The iterations will start from threshold computations 
4. The objective is to perfrom grid search and monitor warnings in case of full and ablated AEDDM drfit detector

# Dataset 

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing


from skmultiflow.data import HyperplaneGenerator
from scipy.stats import norm
import random
from scipy import stats 


import tensorflow
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import anderson

In [6]:
data=pd.read_csv('covtype.csv')

In [7]:
data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,0.368684,0.141667,0.045455,0.184681,0.223514,0.071659,0.870079,0.913386,0.582677,0.875366,...,0,0,0,0,0,0,0,0,0,5
1,0.365683,0.155556,0.030303,0.151754,0.215762,0.054798,0.866142,0.925197,0.594488,0.867838,...,0,0,0,0,0,0,0,0,0,5
2,0.472736,0.386111,0.136364,0.191840,0.307494,0.446817,0.921260,0.937008,0.531496,0.853339,...,0,0,0,0,0,0,0,0,0,2
3,0.463232,0.430556,0.272727,0.173228,0.375969,0.434172,0.937008,0.937008,0.480315,0.865886,...,0,0,0,0,0,0,0,0,0,2
4,0.368184,0.125000,0.030303,0.109520,0.222222,0.054939,0.866142,0.921260,0.590551,0.860449,...,0,0,0,0,0,0,0,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,0.268634,0.425000,0.303030,0.060845,0.245478,0.015175,0.944882,0.933071,0.464567,0.116688,...,0,0,0,0,0,0,0,0,0,3
581008,0.266133,0.422222,0.287879,0.047960,0.239018,0.013348,0.944882,0.933071,0.468504,0.117803,...,0,0,0,0,0,0,0,0,0,3
581009,0.263632,0.441667,0.257576,0.042949,0.232558,0.012646,0.929134,0.948819,0.511811,0.119058,...,0,0,0,0,0,0,0,0,0,3
581010,0.262631,0.472222,0.227273,0.042949,0.229974,0.012646,0.905512,0.964567,0.562992,0.120452,...,0,0,0,0,0,0,0,0,0,3


In [8]:
# Merging all other classes except class 2 into class 1 to make a binary class problem
data["class"] = np.where(data["class"] ==2,2,1)

In [9]:
data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,0.368684,0.141667,0.045455,0.184681,0.223514,0.071659,0.870079,0.913386,0.582677,0.875366,...,0,0,0,0,0,0,0,0,0,1
1,0.365683,0.155556,0.030303,0.151754,0.215762,0.054798,0.866142,0.925197,0.594488,0.867838,...,0,0,0,0,0,0,0,0,0,1
2,0.472736,0.386111,0.136364,0.191840,0.307494,0.446817,0.921260,0.937008,0.531496,0.853339,...,0,0,0,0,0,0,0,0,0,2
3,0.463232,0.430556,0.272727,0.173228,0.375969,0.434172,0.937008,0.937008,0.480315,0.865886,...,0,0,0,0,0,0,0,0,0,2
4,0.368184,0.125000,0.030303,0.109520,0.222222,0.054939,0.866142,0.921260,0.590551,0.860449,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,0.268634,0.425000,0.303030,0.060845,0.245478,0.015175,0.944882,0.933071,0.464567,0.116688,...,0,0,0,0,0,0,0,0,0,1
581008,0.266133,0.422222,0.287879,0.047960,0.239018,0.013348,0.944882,0.933071,0.468504,0.117803,...,0,0,0,0,0,0,0,0,0,1
581009,0.263632,0.441667,0.257576,0.042949,0.232558,0.012646,0.929134,0.948819,0.511811,0.119058,...,0,0,0,0,0,0,0,0,0,1
581010,0.262631,0.472222,0.227273,0.042949,0.229974,0.012646,0.905512,0.964567,0.562992,0.120452,...,0,0,0,0,0,0,0,0,0,1


In [10]:
from sklearn.utils import shuffle
data = shuffle(data).reset_index(drop=True)

In [11]:
#drifted_data=pd.read_csv('stream_top25.csv')

In [12]:
#del drifted_data['class']

In [13]:
#drifted_data=drifted_data[0:1280]

In [14]:
#drifted_data = drifted_data.drop(['Unnamed: 0'], axis=1) 

### Add Noise

In [15]:
def add_noise(df,std_dev=0.03):
    df_noisy=df.copy()
    for col in df.select_dtypes(include=['float64','int64']).columns:
        noise=np.random.normal(0,std_dev,size=len(df))
        df_noisy[col] = df_noisy[col] + noise
    return df_noisy


In [16]:
#drifted_data = add_noise(drifted_data, std_dev=0.03) 

## 3. Data Preprocessing 
1. Initial 70% data is used for training ( training and validation for AE)
2. Next 20 % data is used as a validation set to compute thresholds
3. Next 10 % data is used as a test stream initilally without drift 
  

In [17]:
def normalize_encode_split(data,label_col,pos_val,neg_val):        
    # Apply Label Encoding
    
    for col in data.columns:
        if ((data[col].dtype=='object')and (col!=label_col)): 
            data = pd.get_dummies(data, columns=[col])
            
   # Apply Minmax Normalization 
    for col in data.columns:
        if (((data[col].dtype=='float64')or(data[col].dtype=='int64')) and (col!=label_col)): 
            data[col] = np.round((data[col] - data[col].min()) / (data[col].max() - data[col].min()) ,3)  
            
            
    # Split into training  , test (validation set 1) and stream ( drifted data stream)   
            
    train=data[0:int(len(data)*0.70)]
    test=data[int((0.70*len(data))):int((0.90*len(data)))].reset_index(drop=True)
    stream=data[int((0.90*len(data))):len(data)].reset_index(drop=True)
    
    train_positives = train[train[label_col] == pos_val]
    train_negatives = train[train[label_col] == neg_val]
    
    X_positive=train_positives.drop([label_col],axis=1)
    X_negative=train_negatives.drop([label_col],axis=1)
    return train, test, X_positive,X_negative , stream      

In [18]:
train, test, X_positive,X_negative , stream =normalize_encode_split(data,'class',1,2)

In [19]:
#train, test, X_positive,X_negative , stream, y_positive ,y_negative =normalize_encode_split(data,'output',1,0)

In [20]:
drifted_data=stream.copy()
del drifted_data['class']
drifted_data=drifted_data[0:1280]
drifted_data = add_noise(drifted_data, std_dev=0.03) 

### Best Autoencoder Hyperparameters will be deternmined for each data set separetely 

In [21]:
drifted_data

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40
0,0.448892,0.045725,0.243857,0.221516,0.310647,0.289049,0.828839,0.784823,0.591742,0.440039,...,0.045394,0.013436,-0.019759,0.010070,0.037915,0.012368,-0.014512,0.022651,-0.013782,-0.032438
1,0.474906,0.517139,0.185833,0.241292,0.480166,0.268254,0.813239,0.976391,0.633285,0.114885,...,0.021050,-0.031289,-0.027670,0.004251,0.000730,0.012683,-0.006752,0.009500,0.019201,0.028012
2,0.632451,0.043636,0.146010,0.134667,0.253487,0.627203,0.835458,0.877751,0.542766,0.339155,...,-0.041215,0.002666,0.008492,-0.015089,-0.054081,-0.020675,0.003419,-0.032089,-0.017084,0.055752
3,0.607223,0.311351,0.176633,0.473178,0.201886,0.119229,0.919034,0.934611,0.468792,0.144601,...,0.007624,0.006981,0.013085,0.042651,-0.007558,0.017719,-0.022607,-0.005248,-0.000066,0.038894
4,0.528488,0.933990,0.181493,0.046092,0.279088,0.586925,0.735801,0.933021,0.744326,0.830614,...,-0.003802,0.000340,-0.041552,-0.056673,-0.016688,-0.014705,0.018268,-0.018163,0.027924,-0.030413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275,0.539572,0.181792,0.392338,0.010146,0.275395,0.087518,0.906810,0.712091,0.278041,0.324153,...,-0.028423,-0.015102,1.069391,0.023936,0.017036,0.008026,0.033035,0.030471,-0.087439,0.009976
1276,0.599314,0.449131,0.313119,0.550514,0.416029,0.197716,0.841360,0.896600,0.481843,0.073167,...,0.015039,0.011185,0.008547,-0.024814,-0.010945,-0.008798,0.000816,-0.028088,-0.015780,-0.030437
1277,0.470736,0.337706,0.088192,0.163147,0.253053,0.286700,0.923773,0.923800,0.488425,0.333919,...,-0.025676,0.010346,-0.026271,0.007412,-0.003948,0.028131,0.038539,-0.005293,-0.044271,0.024674
1278,0.594475,0.965878,-0.016866,0.137037,0.191607,0.547669,0.820780,0.922256,0.598423,0.477774,...,0.029053,-0.033514,-0.058377,-0.074936,0.012775,-0.024721,0.002202,-0.050047,-0.046197,-0.009512


In [22]:
def autoencoder (train,epochs,val_set):
    
    # set bottleneck 1/3 of the input layer size
    #encoding_dim = int(np.floor(train.shape[-1]/3))
       
    # input layer size= # of attributes in the dataset after one-hot encoding
    input_layer = tensorflow.keras.Input(shape=train.shape[1:]) # Input Layer
    encoded = layers.Dense(18, activation='relu')(input_layer) # Code Layer 1
    encoded=layers.Dense(6,activation='relu')(encoded) # Bottleneck
    decoded=layers.Dense(18,activation='relu')(encoded)# Decode Layer 1
    decoded = layers.Dense(train.shape[1], activation='sigmoid')(decoded) # Output Layer
    
      
    autoencoder = tensorflow.keras.Model(input_layer, decoded)
    # above model maps an input to its reconstruction 
    
            
    autoencoder.compile(optimizer='adam', loss='mse') # Train autoencoder 
    history=autoencoder.fit(train,train, 
                epochs=epochs,
                batch_size=32,
                shuffle=True,
                validation_data=(val_set, val_set)).history
    return  autoencoder , history

In [23]:
def train_encoders(X_Positive,X_Negative, epochs):
    
    X_Positive_train=X_Positive[0:int(len(X_Positive)*0.90)]
    X_Positive_test=X_Positive[int((0.90*len(X_Positive))):len(X_Positive)-1]
    
    X_Negative_train=X_Negative[0:int(len(X_Negative)*0.90)]
    X_Negative_test=X_Negative[int((0.90*len(X_Negative))):len(X_Negative)-1]

    print("Training Autoencoder on Positive Examples ")
    encoder_pos_class, history_positive_class=autoencoder(X_Positive_train,epochs,X_Positive_test)
    print("Training Autoencoder on Negative Examples ")
    encoder_neg_class,history_negative_class=autoencoder(X_Negative_train,epochs,X_Negative_test)
    
    return encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class

In [24]:
encoder_pos_class, history_positive_class ,encoder_neg_class,history_negative_class=train_encoders(X_positive,X_negative, 100)

Training Autoencoder on Positive Examples 
Epoch 1/100
5864/5864 [==============================] - 16s 3ms/step - loss: 0.0277 - val_loss: 0.0046
Epoch 2/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0045 - val_loss: 0.0036
Epoch 3/100
5864/5864 [==============================] - 15s 3ms/step - loss: 0.0036 - val_loss: 0.0034
Epoch 4/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0033 - val_loss: 0.0028
Epoch 5/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0028 - val_loss: 0.0024
Epoch 6/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0024 - val_loss: 0.0023
Epoch 7/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0023 - val_loss: 0.0023
Epoch 8/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0022 - val_loss: 0.0021
Epoch 9/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0020 - val_loss: 0.0020:
Epoch 10/100
5864/58

5864/5864 [==============================] - 14s 2ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 78/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0010 - val_loss: 0.0011TA: 0s - loss: 0.00
Epoch 79/100
5864/5864 [==============================] - 13s 2ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 80/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 81/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0010 - val_loss: 0.0011
Epoch 82/100
5864/5864 [==============================] - 14s 2ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 83/100
5864/5864 [==============================] - 13s 2ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 84/100
5864/5864 [==============================] - 13s 2ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 85/100
5864/5864 [==============================] - 13s 2ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 86/100
5864/5864 [=========================

5575/5575 [==============================] - 11s 2ms/step - loss: 0.0026 - val_loss: 0.0026ss: - ETA: 2s - loss: 0. - ETA: 2s - l - ETA: 2s - loss: 0.002 - ETA: 2s - loss: 0. - ETA: 0s - loss: 0.002 - ETA: 0s - l
Epoch 23/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0026 - val_loss: 0.00258s - loss: 0.0 - ETA: 8s - loss - ETA: 7s - loss: 0 -  - ETA: 6s - los - ET - ETA:  - ETA: 4s - los - ETA - ETA: 2s - loss: - ETA: 2s - los - ETA: 1s - l - ETA: - ETA: 0s - lo
Epoch 24/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0026 - val_loss: 0.0025- ETA: 9s - loss: 0.00 - ETA: 8s - loss:  - - ETA: 7s - loss: 0.0  - ETA: 6s - - ETA: 5s - los - ETA: 4s - l - ETA:  - ETA: 0s - loss: 0.002 - ETA: 0s - loss - ETA: 0s - loss: - ETA: 0s - loss: 0.
Epoch 25/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0026 - val_loss: 0.0025loss: - ETA: 2s  - ETA: 1s - loss:  - ETA: 1s - loss: 0 - ETA: 1s - loss: 0 - ETA: 1s - loss:  - ETA: 

5575/5575 [==============================] - 11s 2ms/step - loss: 0.0021 - val_loss: 0.0021ETA: 3 - ETA: 2s - loss: 0.0 - ETA: 2s - loss: 0.002 - ETA: - ETA: 1s - loss: 0.0 - ETA: 1s - loss: 0 - ETA: 0s - loss: 0.0
Epoch 52/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0021 - val_loss: 0.0021TA: 9s - loss: 0.0 - ETA: 9s - ETA: 8s - ETA: 7s - loss:  - ETA: 7s - los - ETA: 6s - loss: - ETA: 6s - loss: 0.002 - ETA: 6s - loss - ETA: 6s - loss -  - ETA: 4s  - ETA: 3s - loss: 0.00 - ETA: 3s - loss: - ETA: 3s - loss: 0.002 - ETA: 3s - loss: 0.0 - ETA: 3s - loss: 0 - ETA: 2s - loss - ETA: 2s  - ETA: 1s -   - ETA: 0s - loss: 0.002
Epoch 53/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0021 - val_loss: 0.0020- ETA: 1s  - ETA: 0s - loss: 0.00 - ETA: 0s - loss: - ETA: 0s - loss: 0.002 - ETA: 0s - loss: 0.
Epoch 54/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0021 - val_loss: 0.0020ETA: 5s - loss: 0.00 - ETA: 5s - - E - 

5575/5575 [==============================] - 11s 2ms/step - loss: 0.0021 - val_loss: 0.0020loss: 0 - ETA: 8s - loss: 0 - ETA: 8s - loss: 0. - ET -  - ETA: 5s - loss: 0.00 - ETA: 4s - los - ETA: 4s - loss: 0.00 - ETA: 4s - l - ETA:  - ETA: 2s -  - ETA: 1s - loss: 0. - ETA: 0s - - ETA: 0s - loss: 0.00 - ETA: 0s - loss: 0.002
Epoch 81/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0020 - val_loss: 0.0019- ETA: 9s - loss: 0 - ETA: 8s - loss: 0. - ETA - ETA: 7s  - ETA: 7s - loss: 0. - ETA: 6s - - ETA: 6s - loss: 0.0 - ETA: 3s - lo - ETA: 2s - loss - ETA: 2s - loss:  - E 
Epoch 82/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0020 - val_loss: 0.0020- ETA: 6s - loss: 0.0 - ETA: 5s - loss - ETA: 4s - loss: 0.0 - ETA: 4s - loss: - ETA: 3s - loss: 0.002 - ETA: 3s - loss: 0.0 - ETA: 3s - loss: - ETA: 0s - l
Epoch 83/100
5575/5575 [==============================] - 11s 2ms/step - loss: 0.0020 - val_loss: 0.0020ss: 0. - ETA: 4s - l - ETA: 4s - loss

In [25]:
test

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,class
0,0.658,0.922,0.227,0.000,0.224,0.585,0.728,0.870,0.685,0.269,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.593,0.481,0.348,0.204,0.258,0.335,0.894,0.961,0.531,0.438,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
2,0.472,0.339,0.167,0.122,0.244,0.334,0.937,0.913,0.484,0.293,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
3,0.543,0.097,0.288,0.653,0.336,0.343,0.843,0.768,0.429,0.155,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
4,0.545,0.772,0.030,0.271,0.269,0.549,0.843,0.941,0.638,0.643,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116197,0.062,0.928,0.500,0.219,0.388,0.039,0.508,0.701,0.701,0.131,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
116198,0.499,0.911,0.136,0.064,0.227,0.394,0.783,0.906,0.669,0.333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
116199,0.509,0.433,0.136,0.219,0.253,0.721,0.906,0.949,0.563,0.582,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2
116200,0.661,0.106,0.045,0.261,0.258,0.228,0.862,0.917,0.591,0.039,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


## Validation Data

In [26]:
def make_batches(test_data): 
        data=np.array(test_data)
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches

In [27]:
def process_validation_data(test,encoder_pos_class,encoder_neg_class,batch_size=32):
   
    # From the test data ,separate positive and negative class data and remove the class column

    test_pos_class=test[test['class']==2]
    test_neg_class=test[test['class']==1]
    del test_pos_class['class']
    del test_neg_class['class']
       
       
    # This function make batches of the data stream
     
    def make_batches(test_data):
        data=np.array(test_data)
        batch_size=32
        batches={}
        count=0
        for index in range(0,data.shape[0],batch_size):
            batches[count]=data[index:min(index+batch_size,data.shape[0]),:]
            count+=1
        return batches
    #
    
    batches_pos=make_batches(test_pos_class)
    batches_neg=make_batches(test_neg_class)
    
    # This function computes reconconstruction error for each instance as well as average reco.error of a batch passed to it. 
    def compute_instance_loss_batch_loss(batch,batch_size,encoder):
        mse_list=[]
        mse_sum=0  
        for i in range(0,batch.shape[0]):
            ROW = np.array([batch[i]])
            pred= encoder.predict(ROW) 
            mse = np.mean(np.power(batch[i] - pred, 2))
            mse_list.append(mse)
            mse_sum+=mse
        avg_mse=mse_sum/batch_size
        return mse_list,avg_mse   

            
    # This function computes recon.errr of all the batches .  
    def process_all_batches(batches,encoder):
        batch_avg_mse=[]
        batch_mse_values={}
        for b in batches:
            print(b,end=" ")
            mse_list,average_mse=compute_instance_loss_batch_loss(batches[b],batch_size,encoder)
            batch_avg_mse.append(average_mse)
            batch_mse_values[b]=mse_list
        return batch_avg_mse,batch_mse_values

    batch_avg_mse_pos,batch_mse_values_pos=process_all_batches(batches_pos,encoder_pos_class)
    batch_avg_mse_neg,batch_mse_values_neg=process_all_batches(batches_neg,encoder_neg_class)
       
    
    
    return  batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg
    
    
    
   
    

In [28]:
len(test)

116202

In [29]:
test=test[0:50000]

In [30]:
batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg=process_validation_data(test,encoder_pos_class,encoder_neg_class)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [31]:
k_parameter=list(np.arange(0.5, 4.5, 0.25))
alpha_parameter=['median','max']
logical_parameter=['and','or']
import itertools

hyperparameter_space = {
    'k': k_parameter,
    'count_threshold_measure':alpha_parameter,
    'logical_parameter':logical_parameter
   
}

# Get all the possible combinations of hyperparameters using the `itertools.product()` function
all_combinations = list(itertools.product(*hyperparameter_space.values()))
all_combinations;

In [32]:
len(all_combinations)

64

### AEDDM Framwework

In [33]:
def detect_at_batch_level(test_batch,b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list):
    
    # Layer 1 Variables 
    mse_list_layer1=[]     # Holds the recon loss values predicted by Layer 1 Autoencoder for a single batch
    exceed_count_layer1=0  # How many instances exceed layer one instance level threshold
    mse_sum=0              # sum of recon.error values from layer 1 AE for this batch
    
    
    
    # Layer 2 Variables 
    mse_list_layer2=[]      # Holds the recon.error values predicted by Layer 2 Autoencoder for a single batch
    exceed_count_layer2=-1  # If a batch is not passed to the layer2 AE , then exceed count value will be -1 . This is symbolic.
    mse_sum_layer2=0        # sum of recon.error values from layer 2 AE for this batch
    
        
    layer1_excede_list=[] # Holds the batch numbers of bathces exceeding layer1 thresholds
    #all_excede_list=[]    # Hold the batch number of batches exceeding both Layer-1 and Layer-2 AE Thresholds 
    layer_one_instance_exceed_list=[] # Holds the indices of instances exceeding layer 1 instance threshold
    layer_two_instance_exceed_list=[] # Holds the indices of instances exceeding layer 2 instance thresholds
     
      
    # Determine Layer 1 and Layer AE and their associated thresholds
    layer_one_batch_thres= batch_thres_pos if batch_thres_pos<batch_thres_neg  else batch_thres_neg
    layer_two_batch_thres= batch_thres_pos if batch_thres_pos>batch_thres_neg  else batch_thres_neg
    layer_one_encoder= encoder_pos_class if batch_thres_pos<batch_thres_neg else encoder_neg_class
    layer_two_encoder= encoder_pos_class if batch_thres_pos>batch_thres_neg else encoder_neg_class
    
    layer_one_count_threshold=count_thresh_pos if batch_thres_pos<batch_thres_neg  else count_thres_neg
    layer_two_count_threshold=count_thresh_pos if batch_thres_pos>batch_thres_neg  else count_thres_neg
    
    layer1_ins_thresh=instance_thresh_pos if  batch_thres_pos<batch_thres_neg  else instance_thresh_neg
    layer2_ins_thresh=instance_thresh_pos if  batch_thres_pos>batch_thres_neg  else instance_thresh_neg
    
    # Pass each instance of a batch to Layer 1 AE. Compute Batch MSE and Number of Instances excceding Layer 1 Instance Thresh       
    avg_mse_layer1=0
    for i in range(0,test_batch.shape[0]):
        ROW = np.array([test_batch[i]])
        pred= layer_one_encoder.predict(ROW) 
        mse = np.mean(np.power(test_batch[i] - pred, 2))
        mse_list_layer1.append(mse)
        if mse>layer1_ins_thresh:
            exceed_count_layer1+=1
            layer_one_instance_exceed_list.append(i)
        mse_sum+=mse    
    avg_mse_layer1=(mse_sum)/len(test_batch)
    avg_mse_layer2=0 
    # Check if This batch exceeds both Layer 1 batch and count thresholds
    if ((avg_mse_layer1>layer_one_batch_thres) and ( exceed_count_layer1 >layer_one_count_threshold) ):
        layer1_excede_list.append(b)# Keep track of batches exceesding layer 1 thresholds
        exceed_count_layer2=0
        # Pass each instance of this batch to Layer 2 AE. Compute Batch MSE and Number of Instances excceding Layer 2 Instance Thresh
        for i in range(0,test_batch.shape[0]):
            ROW = np.array([test_batch[i]])
            pred= layer_two_encoder.predict(ROW) 
            mse = np.mean(np.power(test_batch[i] - pred, 2))
            mse_list_layer2.append(mse)
            if mse>layer2_ins_thresh:
                exceed_count_layer2+=1
                layer_two_instance_exceed_list.append(i)
            mse_sum_layer2+=mse    
        avg_mse_layer2=(mse_sum_layer2)/len(test_batch)
        
        if  (avg_mse_layer2 > layer_two_batch_thres) and (exceed_count_layer2>layer_two_count_threshold):
            all_excede_list.append(b)# Keep track of batches exceesding layer 2 thresholds
          
    return  all_excede_list,mse_list_layer1 , exceed_count_layer1 ,avg_mse_layer2,exceed_count_layer2,len(layer_two_instance_exceed_list),layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list,layer_two_instance_exceed_list,avg_mse_layer1 

In [34]:
def detect_stream_drift(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1




In [35]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=float('inf') # No drift in the data
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if(eval(f"({batch_mse[i]} > {Thresh}) {logical} ({exceed_list[i]} > {count_thresh})")):
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [36]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

In [37]:
batch_size=32
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=40):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['class']
#batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
#batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
#drifted=drifted_data[0:1280]
batches_d=make_batches(drifted_data)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    
  
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.25))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [38]:
drift_point=float('inf')

In [39]:
total_batches=40
batch_size=32
drift_point=20

WnDs=[i for i in range(0,21)]
print("Warnings in Drift Stream before drift point",WnDs)
DnDs=[i for i in range(0,21)]
print("Detections in Drift Stream before drift point",DnDs)
delay=[ i for i in range(21)]
print("Detection Delay:",delay)
DdDs=[i for i in range(19)]
print("Detections after drift point:",DdDs)
WdDs=[i for i in range(19)]
print("Warnings after drift point:",WdDs)

Warnings in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections in Drift Stream before drift point [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detection Delay: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Detections after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
Warnings after drift point: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [40]:
WnDS=[i for i in range(0,21)]
DnDs=[i for i in range(0,21)]
delay=[ i for i in range(21)]
DdDs=[i for i in range(19)]
WdDs=[i for i in range(19)]

In [41]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    best_value = float('-inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value > best_value:
                            best_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, best_value

# Find the best parameters and the corresponding value
best_parameters, best_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Best parameters:', best_parameters)
print('Value:', best_value)

Best parameters: (0, 0, 0, 18, 18)
Value: 54


In [42]:
def exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs):
    best_parameters = None
    worst_value = float('inf')

    
    for wn_ds in WnDS:
        for dn_ds in DnDs:
            for d in delay:
                for dd_ds in DdDs:
                    for wd_ds in WdDs:
                        value = wn_ds*(-1)+dn_ds*(-2)+d*(-3)+dd_ds*(2) +wd_ds*(1)

                        if value < worst_value:
                            worst_value = value
                            best_parameters = (wn_ds, dn_ds, d, dd_ds, wd_ds)

    return best_parameters, worst_value

# Find the best parameters and the corresponding value
best_parameters, worst_value = exhaustive_search(WnDS, DnDs, delay, DdDs, WdDs)

# Print the best parameters and the corresponding value
print('Worst parameters:', best_parameters)
print('Value:', worst_value)

Worst parameters: (20, 20, 20, 0, 0)
Value: -120


In [43]:
results_df_full,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.0013867182925724618 , Count Threshold: 4.0
Thresholds for Nagative AE
Batch Thresold: 0.010267150360631883 , Count Threshold: 10.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 6, 12, 17, 19, 20, 21, 23, 25, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 10



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 16, 18, 19, 20, 22, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 4, 7, 9, 12, 13, 16, 17, 18, 19, 20, 22, 27, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thres


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 5, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 5, 6, 10, 13, 14, 19, 20, 23, 25, 26, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 5, 8, 11, 13, 15, 16, 19, 21, 25, 26, 27, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 7, 8, 9, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 5, 13, 15, 16, 17


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 6, 12, 17, 19, 20, 21, 23, 25, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 10



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 16, 18, 19, 20, 22, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 4, 7, 9, 12, 13, 16, 17, 18, 19, 20, 22, 27, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 17



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 15, 17, 18, 19, 20, 21, 23, 24, 26, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 6, 8, 14, 18, 23, 31]

Number of Data Point


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 5, 8, 11, 13, 15, 16, 19, 21, 25, 26, 27, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 7, 8, 9, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 5, 13, 15, 16, 17, 18, 19, 22, 23, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 5, 7, 11, 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 8, 10, 12, 15, 19, 24, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [6, 20]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 5, 6, 7, 15, 16, 20, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 4, 9, 12, 13, 16, 17, 18, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 15, 17, 18, 23, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 5, 10, 11, 13, 17, 18, 19, 21, 22, 28, 29] 

Data Points Exceeding Layer 2 Encoder Inst

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 11, 17, 18, 19, 21, 23, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 32

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 6, 8, 9, 10, 14, 17, 19, 25, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 33

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 5, 7, 8, 9, 10, 12, 14, 22, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 34

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 5, 7, 8, 10, 13, 20, 23, 25, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Nu


Data Points Exceeding Layer 1 Encoder Instance Threshold : [7, 10, 12, 17, 20, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 18

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 10, 16, 18, 19, 22, 24, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 19

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 19, 21, 23, 25, 27, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 20

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 7, 11, 13, 15, 17, 21, 23, 25, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [12]

Number of Data Points Exceeding Layer 2 Encode


Data Points Exceeding Layer 1 Encoder Instance Threshold : [29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [21, 22, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 4

Data Points Exceeding Layer 1 Encoder Instance 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [7] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 36

Data Points Exceeding Layer 1 Encoder Instance Threshold : [20] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 37

Data Points Exceeding Layer 1 Encoder Instance Threshold : [20, 22] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 38

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 8, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 39

Data Points Exceeding Layer 1 Encoder Inst


Data Points Exceeding Layer 1 Encoder Instance Threshold : [21, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 26

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 7, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 27

Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 28

Data Points Exceeding Layer 1 Encoder Instance Threshold : [10] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 29

Data Points Exceeding Layer 1 Encoder I

In [44]:
results_df_full

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,23,23,20,0,0,-9
1,0.50,median,or,32,32,20,0,0,-36
2,0.50,max,and,5,5,20,0,0,45
3,0.50,max,or,27,27,20,0,0,-21
4,0.75,median,and,15,15,20,0,0,15
...,...,...,...,...,...,...,...,...,...
59,4.00,max,or,0,0,20,0,0,60
60,4.25,median,and,0,0,20,0,0,60
61,4.25,median,or,2,2,20,0,0,54
62,4.25,max,and,0,0,20,0,0,60


In [45]:
results_df_p=pd.DataFrame(columns=['k','alpha','logical','WnDs','DnDs','delay','WdDs','DdDs','score'])

### Grid search uisng only one threshold

In [46]:
def compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point):
    if (len(drift_batches_after_drift_point)>=1):
        delay=drift_batches_after_drift_point[0]-drift_point
    
    else:
        delay=len(batch_mse)-drift_point # if drift not detetced then delay is total batches processed after drift point
    return delay


def detect_drift2(batch_mse,exceed_list, Thresh,count_thresh,logical):
    
    
    drift_point=float('inf') # No drift in the data
    WnDs= 0 
    WdDs=0 
    DnDs=0          
    delay=None     
    DdDs=0   
    
    drift_batches_b4_driftpoint=[]
    drift_batches_after_drift_point=[]
    
    
    
    count=0 ## counts the number of consecutive batches exceding threshold
    w_count=0 # Count of elements in Window. Windows contains batch number which excced thresholds
              #threshold 
    w_index_list=[] # Contains indices of batches which excced thresholds
    drift_batches=[]
    for i in range(0,len(batch_mse)):
        
        
        if batch_mse[i] > Thresh:
            print(' Threshold exceeds at batch : {}'.format(i))
                
            if(len(w_index_list)==0 or (i-w_index_list[-1]==1)):
                # Check if w_index_list is empty or its last entry is the previous batch
                w_index_list.append(i)# then append this batch to w_index_list
            count+=1
          
            print(w_index_list)
            if (count>2):# if for more than two consecutive batches  threshold are surpassed , then 
                         # confirm drift
                drift_batch=i-2 # drift point

                print( " Drift Confirmed at Batch No : % d" %drift_batch)
                drift_batches.append(drift_batch)
                if (drift_batch<drift_point):
                    DnDs+=1
                    drift_batches_b4_driftpoint.append(drift_batch)
                    WnDs+=1
                else:
                    DdDs+=1
                    WdDs+=1
                    drift_batches_after_drift_point.append(drift_batch)
            if (len(w_index_list)>=1 and len(w_index_list)<=2):
                
                #w_level=i-len(w_index_list)
                print("Warning Level at Batch",i)
                
        
        else :
            count=0 # reset count 
            if len(w_index_list)<=3:
                w_index_list=[]
    
    
    print("Number of Drifted Batches" + str(len(drift_batches)))
    print("All Drifted Batches: ", drift_batches)
    print("Drifted Batches before Drift Point: ",drift_batches_b4_driftpoint)
    print("Drifted Batches after Drift Point: ",drift_batches_after_drift_point)
    delay=compute_delay(batch_mse,drift_batches,drift_batches_after_drift_point)
    score=WnDs*(-1)+DnDs*(-2)+delay*(-3)+DdDs*(2)+WdDs*(1)+120
    print(f"WnDs={WnDs} ,WdDs={WdDs} , DnDs={DnDs},DdDs={DdDs} , delay= {delay} , value={score}")
    
    col_values=[WnDs,DnDs,delay,WdDs,DdDs,score]
    
    return score,col_values


In [47]:
def detect_stream_drift2(batches,encoder_pos_class,encoder_neg_class,batch_thres_pos,batch_thres_neg, count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg,logical):
    exceed_count_layer2_instance_thresh={} # Holds Number of instances exceeding thresholds of Both Layer 1 and layer 2 AE.
    
    mse_dict_L1={}         # Holds batchwise recon.error values from Layer 1 AE
    exceed_count_L1={}     # Batchwise number of Instances exceeding layer 1 count threshold
    layer_one_instance_exceed_list={} # batch wise list of instances ( indices ) exceeding Layer 1 count threshold
    avg_mse_l1={}
    
    mse_dict_L2={}        # Holds batchwise recon. error  values from Layer 2 AE
    exceed_count_L2={}    # Batchwise number of Instances exceeding layer 2 count threshold
    layer_two_instance_exceed_list={} # For each batch maintains the indices where recon.error exceeds layer 2 threshold
    avg_mse_l2={}         #
    all_excede_list=[]
    n=0
    for b in batches:
        print("\n\n")
        print("**************")
        print('\nBatch Number : {}'.format(b))
        all_excede_list,mse_dict_L1[b],exceed_count_L1[b],avg_mse_l2[b],exceed_count_L2[b],exceed_count_layer2_instance_thresh[b],layer_two_batch_thres,layer_two_count_threshold,layer_one_instance_exceed_list[b],layer_two_instance_exceed_list[b],avg_mse_l1[b]= detect_at_batch_level(batches[b],b,batch_thres_pos,batch_thres_neg,count_thresh_pos,count_thres_neg,instance_thresh_pos,instance_thresh_neg, all_excede_list)
             
        
        print('\nData Points Exceeding Layer 1 Encoder Instance Threshold : {} '.format(layer_one_instance_exceed_list[b]))
        print('\nData Points Exceeding Layer 2 Encoder Instance Threshold: {}'.format(layer_two_instance_exceed_list[b])) 
        print('\nNumber of Data Points Exceeding Layer 2 Encoder Instance thresholds: {}'.format(exceed_count_layer2_instance_thresh[b]))
        
    mse_list_layer1=return_list_of_dict_values(avg_mse_l1)
    exceed_count_list_layer1=return_list_of_dict_values(exceed_count_L1)
    
    
    ## Detect Drift at Batch Level 
    print ("\n Drift Detection at Batch Level\n")
    exceed_list=return_list_of_dict_values(exceed_count_L2)
    mse_list=return_list_of_dict_values(avg_mse_l2)
    score,col_values=detect_drift2(mse_list,exceed_list,layer_two_batch_thres,layer_two_count_threshold,logical)
    
    
    return score,col_values,all_excede_list,exceed_count_layer2_instance_thresh ,exceed_count_L2,avg_mse_l2 ,mse_list_layer1,exceed_count_list_layer1



In [48]:
# This function computes how many instances in a batch exceed instance threshold
def threshold_exceed_count(batch_mse_values, thr):
    exceed_count={}
    for key in batch_mse_values.keys():
        count=0
        list=batch_mse_values[key]
        for a in range(0,len(list)):
            if list[a]>thr:
                count+=1
        exceed_count[key]=count 
    values = exceed_count.values()
    
    return exceed_count

def compute_count_threshold(exceed_count_dict,alpha):
    exceed_list=[]
    for key in exceed_count_dict.keys():
        exceed_list.append(exceed_count_dict[key]) 
    #print(" from with in count threshold")
    #print(alpha)
    method = getattr(np,alpha)  # this enables np.median() or np.max() based on alpha value
    count_threshold=method(exceed_list)    
    return count_threshold
   
def compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse,batch_avg_mse_values,k,alpha,N=40):
        # Computes Instance Threshold
        zscore_list=[]
        for i in range(0,N):
            value_list=batch_avg_mse_values[i]

            mean=np.mean(value_list)
            sigma=np.std(value_list)
            thres_zscore=mean+k*sigma
            zscore_list.append(thres_zscore)
        # Computes batch threshold
        mean_batch=np.mean(batch_avg_mse)
        sigma_batch=np.std(batch_avg_mse)    
        batch_thresh=mean_batch+k*sigma_batch
        
        method = getattr(np,alpha)
        instance_thresh = method(zscore_list)
        
        return  instance_thresh, zscore_list , batch_thresh


    
# Make batches of the normal data stream     
s=stream.copy()
del s['class']
batches_n=make_batches(s)
#print(batches_n)
# For testing purpose choose 20 batches only
batches_n=dict(itertools.islice(batches_n.items(), 20))
    
# Drifted Data Stream
#drifted=drifted_data[0:1280]
batches_d=make_batches(drifted_data)    
    
    
    
def return_list_of_dict_values(d):
    values_list=[]
    for key in d.keys():
        values_list.append(d[key])
    return values_list
    
  
    
def pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg):
    # Define the parametric space
    k_parameter=list(np.arange(0.5, 4.5, 0.25))
    alpha_parameter=['median','max']
    logical_parameter=['and','or']
    results={}
   
    global results_df_p
    row=0
    
    
    for k in k_parameter:
        for alpha in alpha_parameter:
            for logical in logical_parameter:
                print("********************************")
                
                print(k,alpha,logical)
                values=[k, alpha, logical]
                parameters=(k,alpha,logical)
                instance_thresh_neg,zscore_list_neg,batch_thresh_neg=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_neg,batch_mse_values_neg,k,alpha)
                instance_thresh_pos,zscore_list_pos,batch_thresh_pos=compute_instance_threshold_firstN_batches_batch_threshold(batch_avg_mse_pos,batch_mse_values_pos,k,alpha)
                #print(instance_thresh_pos,zscore_list_pos,batch_thresh_pos)
                # Counts the MSE values exceeding threshold in each batch 
                exceed_count_neg=threshold_exceed_count(batch_mse_values_neg,instance_thresh_neg)
                exceed_count_pos=threshold_exceed_count(batch_mse_values_pos,instance_thresh_pos)
                count_thresh_neg=compute_count_threshold(exceed_count_neg,alpha)
                count_thresh_pos=compute_count_threshold(exceed_count_pos,alpha)
                print("Thresholds for Positive AE")
                #print(alpha)
                print(f"Batch Thresold: {batch_thresh_pos} , Count Threshold: {count_thresh_pos}")
                print("Thresholds for Nagative AE")
                print(f"Batch Thresold: {batch_thresh_neg} , Count Threshold: {count_thresh_neg}")
                
                # AEDDM Starts Here 
                
                thres_zscore_batch_pos=batch_thresh_pos
                thres_zscore_batch_neg=batch_thresh_neg
                             
                
                #all_excede_list_n,exceed_count_L2_instThresh_n ,exceed_count_L2_countThresh_n,avg_mse_l2_list_n,mse_list_layer1_n,exceed_count_list_layer1_n=detect_stream_drift(
                    #batches_n,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    #count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
                print(" Drifted Data Processing Starts Here")
                
                score,col_values,all_excede_list_d,exceed_count_L2_instThresh_d ,exceed_count_L2_countThresh_d,avg_mse_l2_list_d,mse_list_layer1_d,exceed_count_list_layer1_d=detect_stream_drift2(
                    batches_d,encoder_pos_class,encoder_neg_class,thres_zscore_batch_pos,thres_zscore_batch_neg, count_thresh_pos,
                    count_thresh_neg,instance_thresh_pos,instance_thresh_neg,logical)
    
        
                results[parameters]=score
                values.extend(col_values)
                results_df_p = results_df_p.append(pd.Series(values, index=results_df_p.columns), ignore_index=True)
                print(results_df_p)
    
    return results_df_p,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d

In [49]:
results_df_ablated,results,exceed_count_L2_countThresh_d,avg_mse_l2_list_d=pipeline(batch_avg_mse_pos,batch_mse_values_pos,batch_avg_mse_neg,batch_mse_values_neg)

********************************
0.5 median and
Thresholds for Positive AE
Batch Thresold: 0.0013867182925724618 , Count Threshold: 4.0
Thresholds for Nagative AE
Batch Thresold: 0.010267150360631883 , Count Threshold: 10.0
 Drifted Data Processing Starts Here



**************

Batch Number : 0

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 6, 12, 17, 19, 20, 21, 23, 25, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 10



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 16, 18, 19, 20, 22, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 4, 7, 9, 12, 13, 16, 17, 18, 19, 20, 22, 27, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thres


Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 3, 4, 5, 6, 7, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 5, 6, 10, 13, 14, 19, 20, 23, 25, 26, 27, 29, 30]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 23

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 5, 8, 11, 13, 15, 16, 19, 21, 25, 26, 27, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 7, 8, 9, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 5, 13, 15, 16, 17


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 22, 23, 24, 25, 26, 27, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [4, 6, 12, 17, 19, 20, 21, 23, 25, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 10



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 15, 16, 18, 19, 20, 22, 24, 26, 27, 28, 29, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 4, 7, 9, 12, 13, 16, 17, 18, 19, 20, 22, 27, 28, 29, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 17



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13, 15, 17, 18, 19, 20, 21, 23, 24, 26, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [1, 6, 8, 14, 18, 23, 31]

Number of Data Point


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 3, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 25, 26, 27, 28, 29, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 2, 5, 8, 11, 13, 15, 16, 19, 21, 25, 26, 27, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 15



**************

Batch Number : 24

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 2, 3, 6, 7, 8, 9, 11, 12, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 3, 5, 13, 15, 16, 17, 18, 19, 22, 23, 27, 28, 29, 30, 31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 16



**************

Batch Number : 25

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 4, 6, 7, 9, 11, 12, 13, 14, 15, 16, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [0, 1, 3, 5, 7, 11, 


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 8, 10, 12, 15, 19, 24, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [6, 20]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 2



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 5, 6, 7, 15, 16, 20, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [2, 4, 9, 12, 13, 16, 17, 18, 28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 9



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 15, 17, 18, 23, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [31]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 5, 10, 11, 13, 17, 18, 19, 21, 22, 28, 29] 

Data Points Exceeding Layer 2 Encoder Inst

...


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 7, 9, 13, 14, 18, 25, 27, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 2, 4, 5, 6, 8, 10, 20, 24, 26, 28] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 13

Data Points Exceeding Layer 1 Encoder Instance Threshold : [2, 6, 8, 12, 15, 25, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 14

Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 5, 8, 10, 12, 14, 15, 17, 25, 27, 29, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceedin


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 8, 10, 12, 15, 19, 24, 26, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 1

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 5, 6, 7, 15, 16, 20, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [16]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 2

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 15, 17, 18, 23, 24, 26, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 3

Data Points Exceeding Layer 1 Encoder Instance Threshold : [3, 5, 10, 11, 13, 17, 18, 19, 21, 22, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data P


Data Points Exceeding Layer 1 Encoder Instance Threshold : [0, 1, 11, 17, 18, 19, 21, 23, 28, 30] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 32

Data Points Exceeding Layer 1 Encoder Instance Threshold : [4, 6, 8, 9, 10, 14, 17, 19, 25, 30, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [15]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 33

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 3, 5, 7, 8, 9, 10, 12, 14, 22, 28, 31] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: [28]

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 1



**************

Batch Number : 34

Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 2, 3, 5, 7, 8, 10, 13, 20, 23, 25, 26, 28, 29] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Nu


Data Points Exceeding Layer 1 Encoder Instance Threshold : [1, 19] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 20

Data Points Exceeding Layer 1 Encoder Instance Threshold : [11, 13, 17] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 21

Data Points Exceeding Layer 1 Encoder Instance Threshold : [11, 15, 17, 19, 26] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 22

Data Points Exceeding Layer 1 Encoder Instance Threshold : [5, 27] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 23

Data Points Exceeding


Data Points Exceeding Layer 1 Encoder Instance Threshold : [] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 9

Data Points Exceeding Layer 1 Encoder Instance Threshold : [11, 13] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 10

Data Points Exceeding Layer 1 Encoder Instance Threshold : [11] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 11

Data Points Exceeding Layer 1 Encoder Instance Threshold : [25] 

Data Points Exceeding Layer 2 Encoder Instance Threshold: []

Number of Data Points Exceeding Layer 2 Encoder Instance thresholds: 0



**************

Batch Number : 12

Data Points Exceeding Layer 1 Encoder Instance Thr

In [50]:
results_df_ablated

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,27,27,20,0,0,-21
1,0.50,median,or,27,27,20,0,0,-21
2,0.50,max,and,27,27,20,0,0,-21
3,0.50,max,or,27,27,20,0,0,-21
4,0.75,median,and,27,27,20,0,0,-21
...,...,...,...,...,...,...,...,...,...
59,4.00,max,or,0,0,20,0,0,60
60,4.25,median,and,0,0,20,0,0,60
61,4.25,median,or,0,0,20,0,0,60
62,4.25,max,and,0,0,20,0,0,60


In [51]:
results_df_full

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,23,23,20,0,0,-9
1,0.50,median,or,32,32,20,0,0,-36
2,0.50,max,and,5,5,20,0,0,45
3,0.50,max,or,27,27,20,0,0,-21
4,0.75,median,and,15,15,20,0,0,15
...,...,...,...,...,...,...,...,...,...
59,4.00,max,or,0,0,20,0,0,60
60,4.25,median,and,0,0,20,0,0,60
61,4.25,median,or,2,2,20,0,0,54
62,4.25,max,and,0,0,20,0,0,60


In [52]:
pd.options.display.max_rows = 100 


In [53]:
results_df_full['WnDs'].mean()

8.46875

In [54]:
results_df_ablated['WnDs'].mean()

9.15625

In [55]:
df1=results_df_full[results_df_full.index % 2 == 0]
df2=results_df_ablated[results_df_ablated.index % 2 == 0]

In [56]:
len(results_df_full)

64

### two tailed test

In [57]:
df1

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,23,23,20,0,0,-9
2,0.50,max,and,5,5,20,0,0,45
4,0.75,median,and,15,15,20,0,0,15
6,0.75,max,and,0,0,20,0,0,60
8,1.00,median,and,23,23,20,0,0,-9
10,1.00,max,and,0,0,20,0,0,60
12,1.25,median,and,17,17,20,0,0,9
14,1.25,max,and,0,0,20,0,0,60
16,1.50,median,and,16,16,20,0,0,12
18,1.50,max,and,0,0,20,0,0,60


In [58]:
df2

,k,alpha,logical,WnDs,DnDs,delay,WdDs,DdDs,score
0,0.50,median,and,27,27,20,0,0,-21
2,0.50,max,and,27,27,20,0,0,-21
4,0.75,median,and,27,27,20,0,0,-21
6,0.75,max,and,27,27,20,0,0,-21
8,1.00,median,and,23,23,20,0,0,-9
10,1.00,max,and,23,23,20,0,0,-9
12,1.25,median,and,17,17,20,0,0,9
14,1.25,max,and,17,17,20,0,0,9
16,1.50,median,and,16,16,20,0,0,12
18,1.50,max,and,16,16,20,0,0,12


In [59]:
from scipy.stats import ttest_rel

t_statistic, p_value = ttest_rel(df1['WnDs'], df2['WnDs'])

# Print the results
print(f"t-statistic: {t_statistic:.3f}")
print(f"p-value: {p_value:.3f}")

# Interpret the results
alpha = 0.50  # Significance level
if p_value > alpha:
    print("The null hypothesis cannot be rejected.")
else:
    print("The null hypothesis can be rejected.")

t-statistic: -3.481
p-value: 0.002
The null hypothesis can be rejected.


### one tailed test

In [60]:
# Perform the one-tailed paired t-test (right-tailed)
t_statistic, p_value = ttest_rel(df2['WnDs'], df1['WnDs'], alternative='greater') 
# Print the results
print(f"t-statistic: {t_statistic:.3f}")
print(f"p-value: {p_value:.3f}")

# **Null Hypothesis (H0):** The ablated detector produces less than or equal to the number of warnings as the full detector.
# **Alternative Hypothesis (H1):** The ablated detector produces significantly more warnings than the full detector.

# Interpret the results
alpha = 0.05  # Significance level
if p_value < alpha:
    print("Reject the null hypothesis. The ablated detector produces significantly more warnings than the full detector.")
else:
    print("Fail to reject the null hypothesis.") 

t-statistic: 3.481
p-value: 0.001
Reject the null hypothesis. The ablated detector produces significantly more warnings than the full detector.
